Load local model

In [4]:
# Local
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from libs.core.tests.unit_tests.output_parsers.test_openai_tools import message

ds_chat = ChatOllama(model="deepseek-r1:7b")

C:\Users\guosh\AppData\Local\Temp\ipykernel_42856\3086055942.py:8: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  ds_chat = ChatOllama(model="deepseek-r1:7b")


Maintain message list

In [5]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful assistant. Answer all questions to the best of your ability. Answer in English.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt | ds_chat
mes_list=    {
        "messages": [
            HumanMessage(
                content="Translate from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
ai_msg = chain.invoke(mes_list)
print(ai_msg.content)

<think>
Alright, so the user asked me to translate "I love programming." into French earlier, and I responded with "J'adore la programmation." Now they're following up with "What did you just say?" Hmm, okay.

First, let's break down what happened. The initial request was a straightforward translation from English to French. I provided the correct phrase. Then, the user is asking about that response. They might be testing my ability or perhaps verifying if I understood correctly.

I need to consider why they're asking this. Maybe they want confirmation that their previous query was handled properly. Or perhaps they're checking for consistency in the language used in our interactions.

Thinking about the context, since it's a friendly exchange, maybe the user is just curious or maybe they made an error themselves and are seeking clarification. Either way, my response should be helpful and positive to maintain a good rapport.

In French, "What did you just say?" translates naturally as "

Memory
聊天消息历史 (ChatMessageHistory)
大多数（如果不是全部）内存模块的核心实用类之一是 ChatMessageHistory 类。
这是一个超轻量级的包装器，它公开了方便的方法来保存人类消息、AI 消息，然后获取它们全部。
ConversationBufferMemory
现在我们展示如何在链中使用这个简单的概念。
我们首先展示 ConversationBufferMemory，它只是 ChatMessageHistory 的一个包装器，可以提取变量中的消息。

In [29]:
from langchain.memory import ChatMessageHistory, ConversationBufferMemory

history = ChatMessageHistory()
history.add_user_message("hi!")
history.add_ai_message("whats up?")
print(history.messages)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up yo?")
memory.load_memory_variables({})

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}), AIMessage(content='whats up?', additional_kwargs={}, response_metadata={})]


{'chat_history': [HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='whats up yo?', additional_kwargs={}, response_metadata={})]}

chat chain   ConversationChain

In [ ]:
from langchain.chains import ConversationChain

conversation = ConversationChain(
    llm=ds_chat,
    # verbose=True,
    memory=ConversationBufferMemory()
)
conversation.predict(input="Hi there!")

In [ ]:
conversation.predict(input="What did you just say?")

在 LLMChain 中使用 Memory 类。在本演示中，我们将添加 ConversationBufferMemory 类，但实际上可以使用任何记忆类。

In [24]:
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, PromptTemplate
template = """You are a chatbot having a conversation with a human.

{chat_history}

Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"],
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(
    llm=ds_chat,
    prompt=prompt,
    # verbose=True,
    memory=memory,
)

llm_chain.predict(human_input="Hi there my friend")

'<think>\nOkay, the user greeted me as "my friend." I should respond warmly to show they feel comfortable. Maybe mention that I\'m here for help and ask how I can assist them today.\n</think>\n\nHello! My friend, how can I assist you today?'

In [25]:
llm_chain.predict(human_input="Not too bad - how are you?")

'<think>\nAlright, so the user greeted me with "Not too bad - how are you?" after I responded to their initial greeting. It\'s a friendly exchange. They seem to be in good spirits and want to know about my well-being.\n\nI should acknowledge their comment positively to maintain a pleasant tone. Then, it\'s time to pivot into offering help since that was the main goal of our conversation—assisting them today.\n\nMaybe say something like, "Thanks for the kind words! I\'m doing well, thank you. How can I assist you today?" That sounds natural and inviting.\n\nLet me make sure the response is friendly but also professional enough to serve their purpose without being too casual or overly formal.\n</think>\n\nThanks for asking! I\'m doing well, thanks to you. How can I assist you today?'

In [ ]:
llm_chain.memory.buffer

LangGraph
The example below shows how to use LangGraph to implement a ConversationChain or LLMChain with ConversationBufferMemory.

In [31]:
import uuid

from IPython.display import Image, display
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = ds_chat.invoke(state["messages"])
    # We return a list, because this will get added to the existing list
    return {"messages": response}

# Define the two nodes we will cycle between
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Adding memory is straight forward in langgraph!
memory = MemorySaver()
app = workflow.compile(
    checkpointer=memory
)

In [32]:
# The thread id is a unique key that identifies
# this particular conversation.
# We'll just generate a random uuid here.
# This enables a single application to manage conversations among multiple users.
thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

input_message = HumanMessage(content="hi! I'm bob")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# Here, let's confirm that the AI remembers our name!
input_message = HumanMessage(content="what was my name?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

<think>

</think>

Hi Bob! How can I assist you today? 😊
================================ Human Message =================================

what was my name?
================================== Ai Message ==================================

<think>
Alright, the user greeted me as "Bob" and then asked, "what was my name?" 

I need to figure out how to respond appropriately. Since they already identified themselves as Bob earlier, it's important to clarify that.

But wait, in our conversation history, I mentioned "Hi! I'm Bob." So maybe they're confirming their identity or seeking more information about me.

They might be curious about my name or prefer to use a different greeting. It could also be a way for them to set the context for future interactions.

I should acknowledge that my name is Bob and offer further a